# Configuration

NOTES: The warnings after the import are referred to the fact that Tensorflow 2.x versions are built to directly look for a GPU in the system. The warning can be forgot if you are not going to use the GPU. 

In [1]:
!source myenv/bin/activate

In [2]:
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set_style('whitegrid')
import IPython.display as ipd
import librosa.display
import numpy as np
import pickle
import scipy
import ipywidgets
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# from livelossplot import PlotLossesKeras
tf.config.list_physical_devices('GPU')

2021-08-30 16:10:23.089688: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-30 16:10:23.089709: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-30 16:10:23.968251: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-30 16:10:23.970543: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-30 16:10:24.044390: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-30 16:10:24.044414: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /pr

[]

In [3]:
%cd ffmpeg-normalize

/home/helemanc/Desktop/Binary_Model/ffmpeg-normalize


# Load Dataframes

In [4]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/pre-processed"

RAV_df = pd.read_csv(os.path.join(preprocess_path,"RAV_df.csv"))
SAVEE_df = pd.read_csv(os.path.join(preprocess_path,"SAVEE_df.csv"))
TESS_df = pd.read_csv(os.path.join(preprocess_path,"TESS_df.csv"))
CREMA_df = pd.read_csv(os.path.join(preprocess_path,"CREMA_df.csv"))

# Audio Normalization

In [ ]:
!git clone https://github.com/slhck/ffmpeg-normalize

In [ ]:
!python3 setup.py install --user

In [5]:
from pydub import AudioSegment
from pydub.utils import make_chunks

concatenation_tmp_folder = '/home/helemanc/Desktop/Binary_Model/concatenation_audio'
normalized_files_folder = '/home/helemanc/Desktop/Binary_Model/normalized_audio/crema'
ravdess_folder = "/media/helemanc/OS/Users/i2CAT/Desktop/Datasets\ SER/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
savee_folder = "/media/helemanc/OS/Users/i2CAT/Desktop/Datasets\ SER/savee"
tess_folder = "/media/helemanc/OS/Users/i2CAT/Desktop/Datasets\ SER/tess/TESS Toronto emotional speech set data"
crema_folder = "/media/helemanc/OS/Users/i2CAT/Desktop/Datasets\ SER/creamd" 




def audio_normalization(dataset_folder, df):
    paths_normalized = []
    from tqdm import tqdm_notebook
    for path in tqdm_notebook(list(df.path)):
        samples, sr = librosa.load(path, res_type='kaiser_fast', sr=16000)
        duration = librosa.get_duration(y = samples, sr = sr)

        part = path.split('/')
        flag_duration = 0 

        if duration < 3: 
            flag_duration = 1 
            audio = AudioSegment.from_file(path)
            combined_sounds = audio+audio+audio
            conc_path = os.path.join(concatenation_tmp_folder, part[-1])
            combined_sounds.export(conc_path, format="wav") 
        if flag_duration == 0:  # audio file duration > 3s
            new_path = os.path.join(normalized_files_folder, part[-1])
            paths_normalized.append(new_path)
            old_path = os.path.join(dataset_folder, part[-2], part[-1])
            command = "ffmpeg-normalize " + old_path + " -o " +  new_path +   " -ar 16000"
            os.system(command)
        else: 
            new_path = os.path.join(normalized_files_folder, part[-1])
            paths_normalized.append(new_path)
            old_path = os.path.join(concatenation_tmp_folder, part[-1])
            command = "ffmpeg-normalize " + old_path + " -o " +  new_path +   " -ar 16000"
            os.system(command)

            audio_to_cut = AudioSegment.from_file(new_path , "wav") 
            chunk_length_ms = duration*1000 #ms
            chunks = make_chunks(audio_to_cut, chunk_length_ms) # divide the audio file to the original length
            for i, chunk in enumerate(chunks):
                chunk_name = new_path
                chunk.export(chunk_name, format="wav")
                break
    return paths_normalized

In [10]:
rav_path_normalized = audio_normalization(ravdess_folder, RAV_df)

  0%|          | 0/1440 [00:00<?, ?it/s]

exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/03-01-08-01-02-02-11.wav


exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/03-01-02-01-02-02-13.wav


In [11]:
savee_path_normalized = audio_normalization(savee_folder, SAVEE_df)

  0%|          | 0/480 [00:00<?, ?it/s]

exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_sa07.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_sa08.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_sa09.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su02.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su04.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su05.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su06.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su07.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su09.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_su11.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_n17.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/KL_n18.wav
exporting /home/helemanc/De

exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_n18.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_n26.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_sa03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_h13.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_n03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_f03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_h03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_d03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/JE_d07.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/DC_su03.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/DC_su05.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/DC_su06.wav
exporting /home/helemanc/Desktop/Binary_Model/normalized_audio/DC_su11.wav
exporting /home/helemanc/Desktop/

In [ ]:
tess_path_normalized = audio_normalization(tess_folder, TESS_df)

In [6]:
crema_path_normalized = audio_normalization(crema_folder, CREMA_df)

  0%|          | 0/440 [00:00<?, ?it/s]

In [7]:
CREMA_df.shape

(440, 4)